In [1]:
#import tensorflow as tf
#from keras.backend.tensorflow_backend import set_session

#config = tf.ConfigProto()
#config.gpu_options.allow_growth = True
#sess = tf.Session(config=config)
#set_session(sess)

In [2]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)#

    except RuntimeError as e:
        print(e)

In [3]:
"""

This is for morphological classification of galaxies by CNN,
By Kenji Bekki, on 2017/11/15
Revised on 2020/2/14 (Nair & Abraham 2010)
For test only.

"""

from tensorflow.keras.models import model_from_json
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
%matplotlib inline
mpl.rcParams['figure.figsize'] = [10, 10]
mpl.rcParams['figure.dpi'] = 72

import sys
sys.path.insert(0, "..\\utility")
from data_loading import *
count = 1000
#x, Y, metadata = load_hdf5_data(name="nair_abraham_2010_hdf5")
x, Y, metadata = load_hdf5_data(name="nair_200_hdf5")


#TTypes = load_attribute(data="NAIR_ABRAHAM_2010")

#EInd = np.where(TTypes == -5)[0] # Elliptical

#x = x[EInd]

#np.savetxt("Nair_Abraham_2010_Ellipticals.dat", x.flatten())

E:\Anaconda3\envs\hon-tf-2.5.0\lib\site-packages\tables\path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'NYU-DR4_Object_Tag'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
E:\Anaconda3\envs\hon-tf-2.5.0\lib\site-packages\tables\path.py:155: NaturalNameWarning: object name is a Python keyword: 'class'; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
E:\Anaconda3\envs\hon-tf-2.5.0\lib\site-packages\tables\path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'g-r'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)


(2723, 100, 100)


KeyError: 'gclass'

In [ ]:
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.figsize'] = [10, 10]
mpl.rcParams['figure.dpi'] = 72
import plotting_helpers
plotting_helpers.plot_classification_results(images=x, y_preds=Y, y_trues=Y, y_labels=("No Disc", "Disc"), random_sample=True)

In [ ]:
x = x.reshape((-1, 100, 100, 1))
x.shape

In [ ]:
califa_model = tf.keras.models.load_model(r'saved_models\califa_s_1.0');

califa_y_pred_prob = califa_model.predict(x)


In [ ]:
#califa_model = tf.keras.models.load_model(r'saved_models\califa_s_1.0');

#califa_y_pred_prob = califa_model.predict(x)

califa_y_pred = califa_y_pred_prob.argmax(axis=1)
califa_results = np.asarray(np.unique(califa_y_pred, return_counts=True)).T
print("total images: " + str(np.sum(califa_results)-1))
print("Percentage of E: ", califa_results[0, 1]/(np.sum(califa_results)-1)*100)
print("Percentage of ES: ", califa_results[1, 1]/(np.sum(califa_results)-1)*100)

In [ ]:
sim_model = tf.keras.models.load_model(r'saved_models\transfer_learning0.5.h5')

sim_y_pred_prob = sim_model.predict(x)

sim_y_pred = sim_y_pred_prob.argmax(axis=1)
sim_results = np.asarray(np.unique(sim_y_pred, return_counts=True)).T
print("total images: " + str(np.sum(sim_results)-1))
print("Percentage of E: ", sim_results[0, 1]/(np.sum(sim_results)-1)*100)
print("Percentage of ES: ", sim_results[1, 1]/(np.sum(sim_results)-1)*100)

In [ ]:
import plotly.express as px
import pandas as pd
df = pd.DataFrame({'Classifier': ['CNNo', 'CNNt', 'CNNo', 'CNNt'],
                    'Galaxy Type': ['E', 'E', 'ES', 'ES'],
                    'Count': [califa_results[0, 1], sim_results[0, 1], califa_results[1, 1], sim_results[1, 1]]})
fig = px.bar(df, x='Classifier', y='Count', color='Galaxy Type')
fig.show()

In [ ]:
califa_E = np.where(califa_y_pred == 0)[0].tolist()
califa_ES = np.where(califa_y_pred == 1)[0].tolist()

sim_E = np.where(sim_y_pred == 0)[0].tolist()
sim_ES = np.where(sim_y_pred == 1)[0].tolist()

califa_E_and_sim_E = np.intersect1d(califa_E, sim_E).tolist()
califa_E_and_sim_ES = np.intersect1d(califa_E, sim_ES).tolist()
califa_ES_and_sim_E = np.intersect1d(califa_ES, sim_E).tolist()
califa_ES_and_sim_ES = np.intersect1d(califa_ES, sim_ES).tolist()

print("Califa E & Sim E ", len(califa_E_and_sim_E))
print("Califa E & Sim ES ",len(califa_E_and_sim_ES))
print("Califa ES & Sim E ",len(califa_ES_and_sim_E))
print("Califa ES & Sim ES ",len(califa_ES_and_sim_ES))
#fig = px.scatter(df, x=califa_E, y=califa_E)
#fig.show()

In [ ]:
from sklearn import metrics
metrics.ConfusionMatrixDisplay(metrics.confusion_matrix(califa_y_pred, sim_y_pred), display_labels=['CNNo', 'CNNt']).plot(cmap='Blues')

In [ ]:
#print(metadata.columns)
#print(metadata)
#print(metadata.iloc[califa_ES][['Age', 'SFRM', 'n_g']].mean())
#print(metadata.iloc[sim_ES][['Age', 'SFRM', 'n_g']].mean())


#print(metadata.iloc[califa_E][['Age', 'n_g']].mean())

#print(metadata.iloc[califa_ES ][['Age', 'n_g']].mean())

print(metadata.iloc[califa_E]['SFRM'][metadata.iloc[califa_E]['SFRM'] != 999999.0].mean())
print(metadata.iloc[califa_ES]['SFRM'][metadata.iloc[califa_ES]['SFRM'] != 999999.0].mean())


In [ ]:
print(metadata.iloc[califa_E]['Age'][(metadata.iloc[califa_E]['Age'] != 999999.0) & (metadata.iloc[califa_E]['Age'] != 0.0)].mean())
print(metadata.iloc[califa_ES]['Age'][(metadata.iloc[califa_ES]['Age'] != 999999.0) & (metadata.iloc[califa_ES]['Age'] != 0.0)].mean())

print(metadata.iloc[sim_E]['Age'][(metadata.iloc[sim_E]['Age'] != 999999.0) & (metadata.iloc[califa_E]['Age'] != 0.0)].mean())
print(metadata.iloc[sim_ES]['Age'][(metadata.iloc[sim_ES]['Age'] != 999999.0) & (metadata.iloc[califa_ES]['Age'] != 0.0)].mean())
print(metadata.iloc[califa_E]['Age'].describe())

In [ ]:
print(metadata.iloc[califa_E]['Age'][(metadata.iloc[califa_E]['Age'] != 999999.0) & (metadata.iloc[califa_E]['Age'] != 0.0)].describe())
print(metadata.iloc[califa_ES]['Age'][(metadata.iloc[califa_ES]['Age'] != 999999.0) & (metadata.iloc[califa_ES]['Age'] != 0.0)].describe())

In [ ]:
metadata.iloc[califa_E]['Age'][(metadata.iloc[califa_E]['Age'] != 999999.0) & (metadata.iloc[califa_E]['Age'] != 0.0)].hist(bins=6)

In [ ]:
print(metadata.iloc[califa_E]['L_g'].describe())
print(metadata.iloc[califa_E]['Rp50_g'].describe()) # divided by next line
print(metadata.iloc[califa_E]['Rp90_g'].describe())
print(metadata.iloc[califa_E]['Mass'].describe())
print(metadata.iloc[califa_E]['bOverA'].describe())
print(metadata.iloc[califa_E]['Mass_group'].describe())

In [ ]:
i = 0
fig, axs = plt.subplots(3, 2)
fig.suptitle("CALIFA Model Predicted Ellipticals (E)")
metadata['col11_div_col12'] = metadata['Rp50_g'] / metadata['Rp90_g']
for parameter in ['Age', 'L_g', 'col11_div_col12', 'Mass', 'bOverA', 'Mass_group']:
    metadata.iloc[califa_E][parameter][(metadata.iloc[califa_E][parameter] != 999999.0) & (metadata.iloc[califa_E][parameter] > 0.0)].hist(bins=6, legend=True, ax=axs.flatten()[i], color = "skyblue")
    i = i + 1
    #plt.legend()

In [ ]:
i = 0
fig, axs = plt.subplots(3, 2)
fig.suptitle("CALIFA Model Predicted Elliculars (ES)")
for parameter in ['Age', 'L_g', 'col11_div_col12', 'Mass', 'bOverA', 'Mass_group']:
    metadata.iloc[califa_ES][parameter][(metadata.iloc[califa_ES][parameter] != 999999.0) & (metadata.iloc[califa_ES][parameter] > 0.0)].hist(bins=6, legend=True, ax=axs.flatten()[i], color = "skyblue")
    i = i + 1
    #plt.legend()

In [ ]:
i = 0
fig, axs = plt.subplots(3, 2)
fig.suptitle("CALIFA Model ES + E Merged")
for parameter in ['Age', 'L_g', 'col11_div_col12', 'Mass', 'bOverA', 'Mass_group']:
    metadata.iloc[califa_ES][parameter][(metadata.iloc[califa_ES][parameter] != 999999.0) & (metadata.iloc[califa_ES][parameter] > 0.0)].hist(bins=6, legend=True, ax=axs.flatten()[i], color = "red", alpha = 0.5, density=True)
    metadata.iloc[califa_E][parameter][(metadata.iloc[califa_E][parameter] != 999999.0) & (metadata.iloc[califa_E][parameter] > 0.0)].hist(bins=6, legend=True, ax=axs.flatten()[i], color = "skyblue", alpha = 0.5, density=True)
    i = i + 1
    #plt.legend()

In [ ]:
from matplotlib.ticker import PercentFormatter
i = 0
fig, axs = plt.subplots(3, 2)
fig.suptitle("CALIFA Model ES + E Merged")
for parameter in ['Age', 'L_g', 'col11_div_col12', 'Mass', 'bOverA', 'Mass_group']:
    df_ES = metadata.iloc[califa_ES][parameter][(metadata.iloc[califa_ES][parameter] != 999999.0) & (metadata.iloc[califa_ES][parameter] > 0.0)]
    df_E = metadata.iloc[califa_E][parameter][(metadata.iloc[califa_E][parameter] != 999999.0) & (metadata.iloc[califa_E][parameter] > 0.0)]
    axs.flatten()[i].yaxis.set_major_formatter(PercentFormatter(1))
    metadata.iloc[califa_ES][parameter][(metadata.iloc[califa_ES][parameter] != 999999.0) & (metadata.iloc[califa_ES][parameter] > 0.0)].hist(bins=6, legend=True, ax=axs.flatten()[i], color = "red", alpha = 0.5, weights=np.ones(len(df_ES))/len(df_ES))
    metadata.iloc[califa_E][parameter][(metadata.iloc[califa_E][parameter] != 999999.0) & (metadata.iloc[califa_E][parameter] > 0.0)].hist(bins=6, legend=True, ax=axs.flatten()[i], color = "skyblue", alpha = 0.5, weights=np.ones(len(df_E))/len(df_E))
    i = i + 1
    #plt.legend()